# Hands-on demos/exercises 1: Dense matrix manipulation



We compute

$$
y = W x + b
$$

with:
- $ W \in \mathbb{R}^{M \times N} $
- $ x \in \mathbb{R}^{N} $
- $ b \in \mathbb{R}^{M} $

where $M, N \gg 1$, using five different implementations :

1. Pure Python (no NumPy)
2. NumPy
3. CuPy
4. JAX
5. PyTorch [optional]

Credit: everything is generated using ChatGPT 5.2

In [1]:
import time

def format_time(seconds, iters):
    avg = seconds / iters
    return f"total = {seconds:.4f}s, per-iter = {avg*1e3:.3f} ms"


def benchmark(fn, setup_fn, iters=10, warmup=2, sync_fn=None, name=""):
    """
    Generic benchmark helper.

    setup_fn: () -> tuple of arguments for fn
    fn: (*args) -> result
    sync_fn: optional function called as sync_fn(result) to synchronize GPU
    """
    args = setup_fn()

    # Warmup
    for _ in range(warmup):
        y = fn(*args)
        if sync_fn is not None:
            sync_fn(y)

    # Timed runs
    start = time.perf_counter()
    for _ in range(iters):
        y = fn(*args)
        if sync_fn is not None:
            sync_fn(y)
    end = time.perf_counter()

    print(f"{name:>12}: {format_time(end - start, iters)}")


## Implementation using pure python

In [2]:
def py_dense(W, x, b):
    """Compute y = W x + b using pure Python lists."""
    M = len(W)
    y = [0.0] * M
    for i in range(M):
        s = 0.0
        row = W[i]
        # manual dot product
        for j in range(len(x)):
            s += row[j] * x[j]
        y[i] = s + b[i]
    return y


def make_python_data(M, N, seed=0):
    """Random Python lists with shapes:
    - W: M x N
    - x: N
    - b: M
    """
    import random
    random.seed(seed)
    W = [[random.random() for _ in range(N)] for _ in range(M)]
    x = [random.random() for _ in range(N)]
    b = [random.random() for _ in range(M)]
    return (W, x, b)


## Implementation using numpy

In [3]:
try:
    import numpy as np

    def np_dense(W, x, b):
        # W: (M, N), x: (N,), b: (M,)
        return W @ x + b

    def make_numpy_data(M, N, seed=0):
        rng = np.random.default_rng(seed)
        W = rng.random((M, N), dtype=np.float32)
        x = rng.random((N,), dtype=np.float32)
        b = rng.random((M,), dtype=np.float32)
        return (W, x, b)

    HAS_NUMPY = True
except ImportError:
    HAS_NUMPY = False
    print("NumPy not installed; NumPy benchmark will be skipped.")


## Implementation using cupy

In [4]:
try:
    import cupy as cp

    def cp_dense(W, x, b):
        return W @ x + b

    def make_cupy_data(M, N, seed=0):
        cp.random.seed(seed)
        W = cp.random.random((M, N), dtype=cp.float32)
        x = cp.random.random((N,), dtype=cp.float32)
        b = cp.random.random((M,), dtype=cp.float32)
        return (W, x, b)

    def cupy_sync(y):
        # Make sure all kernels are done
        cp.cuda.Stream.null.synchronize()

    # Quick device check
    _ = cp.cuda.Device(0)
    HAS_CUPY = True
except ImportError:
    HAS_CUPY = False
    print("CuPy not installed; CuPy benchmark will be skipped.")
except Exception as e:
    HAS_CUPY = False
    print(f"CuPy found but not usable ({e}); CuPy benchmark will be skipped.")


## Implementation using jax

In [5]:
try:
    import jax
    import jax.numpy as jnp

    @jax.jit
    def jax_dense(W, x, b):
        return W @ x + b

    def make_jax_data(M, N, seed=0):
        key = jax.random.PRNGKey(seed)
        key_W, key_x, key_b = jax.random.split(key, 3)
        W = jax.random.uniform(key_W, (M, N), dtype=jnp.float32)
        x = jax.random.uniform(key_x, (N,), dtype=jnp.float32)
        b = jax.random.uniform(key_b, (M,), dtype=jnp.float32)
        return (W, x, b)

    def jax_sync(y):
        # Ensure computation completes (important on accelerators)
        y.block_until_ready()

    HAS_JAX = True
except ImportError:
    HAS_JAX = False
    print("JAX not installed; JAX benchmark will be skipped.")


In [6]:
try:
    import torch

    def torch_dense(W, x, b):
        return W @ x + b

    def make_torch_data(M, N, seed=0, device="cpu"):
        torch.manual_seed(seed)
        W = torch.rand((M, N), dtype=torch.float32, device=device)
        x = torch.rand((N,), dtype=torch.float32, device=device)
        b = torch.rand((M,), dtype=torch.float32, device=device)
        return (W, x, b)

    def torch_sync(y):
        if y.is_cuda:
            torch.cuda.synchronize()

    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("PyTorch not installed; PyTorch benchmark will be skipped.")


In [7]:
try:
    import torch

    def torch_dense(W, x, b):
        return W @ x + b

    def make_torch_data(M, N, seed=0, device="cpu"):
        torch.manual_seed(seed)
        W = torch.rand((M, N), dtype=torch.float32, device=device)
        x = torch.rand((N,), dtype=torch.float32, device=device)
        b = torch.rand((M,), dtype=torch.float32, device=device)
        return (W, x, b)

    def torch_sync(y):
        if y.is_cuda:
            torch.cuda.synchronize()

    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("PyTorch not installed; PyTorch benchmark will be skipped.")


In [9]:
# ---- Configuration ----
M = 16_384        # rows of W and length of b
N = 16_384        # cols of W and length of x
iters = 10
warmup = 2

# Toggle this to "cuda" if you want to test GPU (and have it available)
torch_device = "cuda"  # or "cpu"

print(f"Benchmarking y = W x + b with:")
print(f"  W: ({M}, {N})")
print(f"  x: ({N},)")
print(f"  b: ({M},)")
print(f"iters = {iters}, warmup = {warmup}")
print("-" * 60)

# 1) Pure Python
try:
    def setup_py():
        return make_python_data(M, N)
    benchmark(py_dense, setup_py, iters=iters, warmup=warmup,
              sync_fn=None, name="pure_python")
except MemoryError:
    print(f"{'pure_python':>12}: skipped (MemoryError - too large M,N)")

# 2) NumPy
if HAS_NUMPY:
    def setup_np():
        return make_numpy_data(M, N)
    benchmark(np_dense, setup_np, iters=iters, warmup=warmup,
              sync_fn=None, name="numpy")

# 3) CuPy
if HAS_CUPY:
    def setup_cp():
        return make_cupy_data(M, N)
    benchmark(cp_dense, setup_cp, iters=iters, warmup=warmup,
              sync_fn=cupy_sync, name="cupy")

# 4) JAX
if HAS_JAX:
    def setup_jax():
        return make_jax_data(M, N)
    benchmark(jax_dense, setup_jax, iters=iters, warmup=warmup,
              sync_fn=jax_sync, name="jax")

# 5) PyTorch
if HAS_TORCH:
    if torch_device == "cuda" and not torch.cuda.is_available():
        print(f"{'torch_cuda':>12}: skipped (CUDA not available)")
    else:
        def setup_torch():
            return make_torch_data(M, N, device=torch_device)
        name = "torch_cuda" if torch_device == "cuda" else "torch_cpu"
        benchmark(torch_dense, setup_torch, iters=iters, warmup=warmup,
                  sync_fn=torch_sync, name=name)


Benchmarking y = W x + b with:
  W: (16384, 16384)
  x: (16384,)
  b: (16384,)
iters = 10, warmup = 2
------------------------------------------------------------
 pure_python: total = 193.5661s, per-iter = 19356.609 ms
       numpy: total = 0.4812s, per-iter = 48.124 ms
        cupy: total = 0.0080s, per-iter = 0.803 ms
         jax: total = 0.0083s, per-iter = 0.832 ms
  torch_cuda: total = 0.0078s, per-iter = 0.784 ms
